In [1]:
import requests
import numpy as np
from bs4 import BeautifulSoup
import pandas as pd

We collect the data on the first page firstly

In [2]:
url = 'https://www.imdb.com/search/title/?title_type=feature&release_date=2019-10-11,2019-11-11&num_votes=1,'

In [3]:
wb_data = requests.get(url)
soup = BeautifulSoup(wb_data.content, "html.parser")
soup.prettify

<bound method Tag.prettify of 
<!DOCTYPE html>

<html xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://ogp.me/ns#">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="app-id=342792525, app-argument=imdb:///?src=mdot" name="apple-itunes-app"/>
<script type="text/javascript">var IMDbTimer={starttime: new Date().getTime(),pt:'java'};</script>
<script>
    if (typeof uet == 'function') {
      uet("bb", "LoadTitle", {wb: 1});
    }
</script>
<script>(function(t){ (t.events = t.events || {})["csm_head_pre_title"] = new Date().getTime(); })(IMDbTimer);</script>
<title>Feature Film,
Released between 2019-10-11 and 2019-11-11,
Rating Count at least 1
(Sorted by Popularity Ascending) - IMDb</title>
<script>(function(t){ (t.events = t.events || {})["csm_head_post_title"] = new Date().getTime(); })(IMDbTimer);</script>
<script>
    if (typeof uet == 'function') {
      uet("be", "LoadTitle", {wb: 1});
    }
</script>
<script>
    if (

In [4]:
item = soup.find_all('h3', class_ = 'lister-item-header')
title = []

for i in item:
    title.append(i.find_all('a'))

In [5]:
movie_title = []
for i in title:
    movie_title.append(i[0].string)

In [6]:
rate = soup.select('.ratings-imdb-rating strong')
rating = []
for i in rate:
    rating.append(i.string)

In [7]:
#test if each column has the same length
print(len(rating))
print(len(movie_title))

50
50


In [38]:
bar  = soup.find_all('div', class_ = 'ratings-bar')
meta = []
for i in bar:
    if i.find('div', class_ = "inline-block ratings-metascore"):
        j = i.find('div', class_ = "inline-block ratings-metascore")
        s = j.find('span').string.strip()
    else:
        s = np.nan
    
    meta.append(s)
    
print(len(meta)) #test length

50


In [47]:
imdb_1 = pd.DataFrame({'title': movie_title, 'rating': rating, 'metascore': meta})
imdb_1

,title,rating,metascore
0,Frozen II,7.0,64
1,Doctor Sleep,7.4,59
2,Midway,6.7,47
3,Charlie's Angels,4.6,52
4,Terminator: Dark Fate,6.3,54
5,Jay and Silent Bob Reboot,5.8,46
6,Maleficent: Mistress of Evil,6.7,43
7,El Camino: A Breaking Bad Movie,7.4,72
8,Last Christmas,6.5,50
9,The Good Liar,6.6,55


Now we need to write codes that handle any number of pages from the result.

We combine codes above to generate some functions for use.

In [48]:
def get_title(url):
    wb_data = requests.get(url)
    soup = BeautifulSoup(wb_data.content, "html.parser")
    item = soup.find_all('h3', class_ = 'lister-item-header')
    title = []

    for i in item:
        title.append(i.find_all('a'))
    movie_title = []
    for i in title:
        movie_title.append(i[0].string)
    return movie_title

In [50]:
def get_rating(url):
    wb_data = requests.get(url)
    soup = BeautifulSoup(wb_data.content, "html.parser")
    rate = soup.select('.ratings-imdb-rating strong')
    rating = []
    for i in rate:
        rating.append(i.string)
    return rating

In [104]:
def get_metascore(url):
    wb_data = requests.get(url)
    soup = BeautifulSoup(wb_data.content, "html.parser")
    bar  = soup.find_all('div', class_ = 'ratings-bar')
    meta = []
    for i in bar:
        if i.find('div', class_ = "inline-block ratings-metascore"):
            j = i.find('div', class_ = "inline-block ratings-metascore")
            s = j.find('span').string.strip()
        else:
            s = np.nan
    
        meta.append(s)
    return meta

In [68]:
def get_next_page(url):
    wb_data = requests.get(url)
    soup = BeautifulSoup(wb_data.content, "html.parser")
    base_url = 'https://www.imdb.com'
    nex = soup.find_all('a', class_ = "lister-page-next next-page")
    return base_url + nex[0]['href']

In [69]:
get_next_page('https://www.imdb.com/search/title/?title_type=feature&release_date=2019-10-11,2019-11-11&num_votes=1,')

'https://www.imdb.com/search/title/?title_type=feature&release_date=2019-10-11,2019-11-11&num_votes=1,&start=51'

In [87]:
def get_multiple_pages_title(n): #The default setting is from the first page.
    url = 'https://www.imdb.com/search/title/?title_type=feature&release_date=2019-10-11,2019-11-11&num_votes=1,'
    title = get_title(url)
    
    while n > 1:
        url = get_next_page(url)
        title = title + get_title(url)
        n = n-1
    return title

In [88]:
def get_multiple_pages_rating(n):
    url = 'https://www.imdb.com/search/title/?title_type=feature&release_date=2019-10-11,2019-11-11&num_votes=1,'
    rating = get_rating(url)
    
    while n > 1:
        url = get_next_page(url)
        rating = rating + get_rating(url)
        n = n-1
    return rating

In [89]:
def get_multiple_pages_meta(n):
    url = 'https://www.imdb.com/search/title/?title_type=feature&release_date=2019-10-11,2019-11-11&num_votes=1,'
    meta = get_metascore(url)
    
    while n > 1:
        url = get_next_page(url)        
        meta = meta + get_metascore(url)
        n = n-1
    return meta

Now we try to make a dataframe which contains 3 pages of movie data.

In [106]:
title3 = get_multiple_pages_title(3)

rating3 = get_multiple_pages_rating(3)

meta3 = get_multiple_pages_meta(3)

In [109]:
imdb3 = pd.DataFrame({'title': title3, 'rating': rating3, 'metascore': meta3})
imdb3

,title,rating,metascore
0,Frozen II,7.0,64
1,Doctor Sleep,7.4,59
2,Midway,6.7,47
3,Charlie's Angels,4.6,52
4,Terminator: Dark Fate,6.3,54
5,Jay and Silent Bob Reboot,5.8,46
6,Maleficent: Mistress of Evil,6.7,43
7,El Camino: A Breaking Bad Movie,7.4,72
8,Last Christmas,6.5,50
9,The Good Liar,6.6,55


In [110]:
imdb3.to_csv('3-page-data from imdb', index=False, sep=',')